# 1. Connecting to Azure and getting the resources from correct Resource Group 

**Before you start you need to login to Azure to correct subscription**

Log into Azure using the Azure CLI:
```powershell
az login --tenant ''
az account set --subscription ''
```

**Setup some variables from Azure that will be used in the code later**

In [ ]:
subscription_id = ''
resource_group_name = ''

connection_string = ""
container_name = ""
storage_account_name = ""

*GitHub Copilot is great with help to install as well the libraries that are missing in the code.*

*To do that ask Copilot to **/fix** the code and you will get the code as below*

*Sometimes Copilot will show code like ```python !pip``` instead of ```python %pip```, but the code will still works for you, as the model is not updated yet with 100% of latest changes of libraries*

In [ ]:
%pip install azure-identity
%pip install azure-mgmt-resource

**GitHub Copilot**

Connect to Azure using Azure Identity and list all the resources from provided resource group. Save the resourceId to the array called 'resource_array'

In [ ]:
from azure.identity import DefaultAzureCredential
from azure.mgmt.resource import ResourceManagementClient

# Authenticate using DefaultAzureCredential
credential = DefaultAzureCredential()

# Initialize the ResourceManagementClient
resource_client = ResourceManagementClient(credential, subscription_id)

# List all resources in the specified resource group
resources = resource_client.resources.list_by_resource_group(resource_group_name)

# Save the resource IDs to the array
resource_array = [resource.id for resource in resources]

print(resource_array)

**GitHub Copilot**

For each resource.id in resource_array, get the json from Azure and save it as a separate file to the folder json_files

In [ ]:
import os
import json

# Ensure the directory exists
os.makedirs('json_files', exist_ok=True)

# Iterate over each resource ID in the resource_array
for resource_id in resource_array:
    # Get the resource details
    resource = resource_client.resources.get_by_id(resource_id, api_version='2021-04-01')
    
    # Convert the resource details to a dictionary
    resource_json = resource.as_dict()
    
    # Create a file name based on the resource name
    file_name = f"json_files/{resource.name}.json"
    
    # Save the resource details to a JSON file
    with open(file_name, 'w') as f:
        json.dump(resource_json, f, indent=4)

print("Resource details saved to json_files directory.")

**GitHub Copilot**

/fix


In [ ]:
import os
import json

# Ensure the directory exists
os.makedirs('json_files', exist_ok=True)

# Iterate over each resource ID in the resource_array
for resource_id in resource_array:
    # Get the resource details
    resource = resource_client.resources.get_by_id(resource_id, api_version='2023-12-01')
    
    # Convert the resource details to a dictionary
    resource_json = resource.as_dict()
    
    # Create a file name based on the resource name
    file_name = f"json_files/{resource.name}.json"
    
    # Save the resource details to a JSON file
    with open(file_name, 'w') as f:
        json.dump(resource_json, f, indent=4)

print("Resource details saved to json_files directory.")

**GitHub Copilot**

If the resource type is 'servers', use api_version: '2014-01-01', otherwise use api_version '2023-12-01'

In [ ]:
import os
import json

# Ensure the directory exists
os.makedirs('json_files', exist_ok=True)

# Iterate over each resource ID in the resource_array
for resource_id in resource_array:
    # Get the resource details
    if 'servers' in resource_id:
        api_version = '2014-01-01'
    else:
        api_version = '2023-12-01'
    resource = resource_client.resources.get_by_id(resource_id, api_version=api_version)
    
    # Convert the resource details to a dictionary
    resource_json = resource.as_dict()
    
    # Create a file name based on the resource name
    file_name = f"json_files/{resource.name}.json"
    
    # Save the resource details to a JSON file
    with open(file_name, 'w') as f:
        json.dump(resource_json, f, indent=4)

print("Resource details saved to json_files directory.")

In [ ]:
import os
import json

# Ensure the directory exists
os.makedirs('json_files', exist_ok=True)

# Iterate over each resource ID in the resource_array
for resource_id in resource_array:
    # Get the resource details
    if 'servers' in resource_id:
        api_version = '2014-01-01'
    else:
        if 'storageAccounts' in resource_id:
            api_version = '2024-01-01'
        else:
            api_version = '2023-12-01'
    resource = resource_client.resources.get_by_id(resource_id, api_version=api_version)
    
    # Convert the resource details to a dictionary
    resource_json = resource.as_dict()
    
    # Create a file name based on the resource name
    file_name = f"json_files/{resource.name}.json"
    
    # Save the resource details to a JSON file
    with open(file_name, 'w') as f:
        json.dump(resource_json, f, indent=4)

print("Resource details saved to json_files directory.")

# 2. Save the files to the blob storage

**GitHub Copilot**

Save all the files from folder json_files to the provided blob storage

In [ ]:
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from azure.core.exceptions import ResourceExistsError

# Initialize the BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# Get the container client
container_client = blob_service_client.get_container_client(container_name)

# Ensure the container exists
try:
    container_client.create_container()
except ResourceExistsError:
    print(f"Container '{container_name}' already exists.")

# Iterate over each file in the json_files folder
for json_file in os.listdir('json_files'):
    file_path = os.path.join('json_files', json_file)
    
    # Create a blob client
    blob_client = container_client.get_blob_client(json_file)
    
    # Upload the file to the blob storage
    with open(file_path, 'rb') as data:
        blob_client.upload_blob(data, overwrite=True)

print("Files uploaded to blob storage.")

# 3. Create a mermaid diagram

Install the required libraries

In [ ]:
%pip install mermaid-py

In [ ]:
import os
import json

# Initialize a markdown string
markdown_content = "# Azure Resources\n\n"

json_files = os.listdir('json_files')

# Iterate over each JSON file in the json_files folder
for json_file in json_files:
    file_path = os.path.join('json_files', json_file)
    
    # Read the JSON file
    with open(file_path, 'r') as f:
        resource_data = json.load(f)
    
    # Extract relevant information
    resource_id = resource_data['id']
    resource_name = resource_data['name']
    resource_type = resource_data['type']
    
    # Append the resource information to the markdown content
    markdown_content += f"## {resource_name}\n"
    markdown_content += f"- **ID**: {resource_id}\n"
    markdown_content += f"- **Type**: {resource_type}\n"
    
    # If the resource has dependencies, list them
    if 'dependsOn' in resource_data:
        markdown_content += "- **Depends On**:\n"
        for dependency in resource_data['dependsOn']:
            dependency_name = dependency.split('/')[-1]
            markdown_content += f"  - {dependency_name}\n"
    
    markdown_content += "\n"

# Save the markdown content to a file
with open('azure_resources.md', 'w') as f:
    f.write(markdown_content)

print("Markdown file 'azure_resources.md' generated.")

In [ ]:
import mermaid as md
from mermaid.graph import Graph

sequence = Graph('Sequence-diagram',"""
stateDiagram-v2
    [*] --> Still
    Still --> [*]

    Still --> Moving
    Moving --> Still
    Moving --> Crash
    Crash --> [*]
""")
render = md.Mermaid(sequence)
render # !! note this only works in the notebook that rendered the html.

**GitHub Copilot**

User the mermaid-py library and 'from mermaid.graph import Graph' to create a graph from files in folder json_files

In [ ]:
import mermaid as md
from mermaid.graph import Graph

# Read the markdown content from the file
with open('azure_resources.md', 'r') as f:
    markdown_content = f.read()

# Initialize the graph with the title and markdown content
graph = Graph(title='Azure Resources', script=markdown_content)

# Render the graph
# graph.show()
render = md.Mermaid(graph)
render

In [ ]:
import mermaid as md
from mermaid.graph import Graph
import os
import json

# Initialize the state diagram
state_diagram = """
stateDiagram-v2
"""

# Iterate over each JSON file in the json_files folder
for json_file in json_files:
    file_path = os.path.join('json_files', json_file)
    
    # Read the JSON file
    with open(file_path, 'r') as f:
        resource_data = json.load(f)
    
    # Extract relevant information
    json_files = os.listdir('json_files')
    resource_name = resource_data['name']
    
    # Add states to the diagram
    state_diagram += f"    {resource_name} --> [*]\n"
    state_diagram += f"    [*] --> {resource_name}\n"

# Create the graph
sequence = Graph('Sequence-diagram', state_diagram)

# Render the graph
render = md.Mermaid(sequence)
render

In [ ]:
import mermaid as md
from mermaid.graph import Graph

sequence = Graph('Sequence-diagram',"""
architecture-beta
    group api(cloud)[API]

    service db(database)[Database] in api
    service disk1(disk)[Storage] in api
    service disk2(disk)[Storage] in api
    service server(server)[Server] in api

    db:L -- R:server
    disk1:T -- B:server
    disk2:T -- B:db
    

""")
render = md.Mermaid(sequence)
render # !! note this only works in the notebook that rendered the html.

# reverse engineer - make a mermaid diagram on web and then write the prompt

In [ ]:
sample_mermaid_diagram = """
architecture-beta
    group api(cloud)[API]

    service db(database)[Database] in api
    service disk1(disk)[Storage] in api
    service disk2(disk)[Storage] in api
    service server(server)[Server] in api

    db:L -- R:server
    disk1:T -- B:server
    disk2:T -- B:db
"""

In [ ]:
new_md = sample_mermaid_diagram

# Iterate over each JSON file in the json_files folder
for json_file in json_files:
    file_path = os.path.join('json_files', json_file)
    
    # Read the JSON file
    with open(file_path, 'r') as f:
        resource_data = json.load(f)
    
    # Extract relevant information
    resource_name = resource_data['name']
    resource_type = resource_data['type'].split('/')[-1]
    
    # Add services to the diagram
    new_md += f"    service {resource_name}({resource_type})[{resource_type}] in api\n"

# Add relationships (example relationships, you can customize as needed)
new_md += """
    db:L -- R:server
    disk1:T -- B:server
    disk2:T -- B:db
"""

# Output the new_md variable
new_md

In [ ]:
import os
import json

# Initialize the state diagram
state_diagram = """
architecture-beta
    group api(cloud)[API]
"""

# Iterate over each JSON file in the json_files folder
json_files = os.listdir('json_files')
for json_file in json_files:
    file_path = os.path.join('json_files', json_file)
    
    # Read the JSON file
    with open(file_path, 'r') as f:
        resource_data = json.load(f)
    
    # Extract relevant information
    resource_name = resource_data['name']
    resource_type = resource_data['type'].split('/')[-1]
    
    # Add services to the diagram
    state_diagram += f"    service {resource_name}({resource_type})[{resource_type}] in api\n"

# Add relationships (example relationships, you can customize as needed)
state_diagram += """
    db:L -- R:server
    disk1:T -- B:server
    disk2:T -- B:db
"""

# Create the graph
sequence = Graph('Sequence-diagram', state_diagram)

# Render the graph
render = md.Mermaid(sequence)
render